In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/train.csv.zip')

Now that we have the data, lets take a closer look at the data. We will look at the distribution of the data as well as the number of missing values and the type of the data.

In [3]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
df.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
toxic_types = ['toxic',	'severe_toxic'	,'obscene',	'threat',	'insult', 	'identity_hate']

#get the value counts for each toxic type
for toxic_type in toxic_types:
    print(df[toxic_type].value_counts())

toxic
0    144277
1     15294
Name: count, dtype: int64
severe_toxic
0    157976
1      1595
Name: count, dtype: int64
obscene
0    151122
1      8449
Name: count, dtype: int64
threat
0    159093
1       478
Name: count, dtype: int64
insult
0    151694
1      7877
Name: count, dtype: int64
identity_hate
0    158166
1      1405
Name: count, dtype: int64


In [17]:
double_toxic=df[(df.toxic==1) & (df.severe_toxic==1)]
double_toxic.shape

(1595, 8)

In [18]:
severely_toxic = df[df.severe_toxic==1]
severely_toxic.shape

(1595, 8)

In [19]:
double_toxic.equals(severely_toxic)

True

Interestignly all the ones tagged as severe_toxic are also labelled as toxic